In [ ]:
#https://github.com/clemtoy/WNAffect
#https://wordnet.princeton.edu/documentation
#https://wordnet.princeton.edu/download/current-version
#https://medium.com/geekculture/simple-emotion-classification-in-python-40fb24692541

In [1]:
from sentic import SenticPhrase
import sentic
import senticnet
import pandas as pd
import seaborn as sns
import re
from textblob import TextBlob
import string
import spacy
import stanza
import nltk
import ssl
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nrclex import NRCLex
from stanfordcorenlp import StanfordCoreNLP
from afinn import Afinn
import statistics

In [2]:
#df = pd.read_csv("datos_train.csv",sep = ',')
import pandas as pd
df = pd.read_excel("datos_features.xlsx")
#df.emotions_NRCLex_categories.head(2)

##### Emotions with NRCLex
Con NRCLex, se usa un lexicon de 10000 a 27000 palabras, basadas en sinónimos de WordNet.
Para cada fila, devuelve las emociones con score más elevado.

https://pypi.org/project/NRCLex/

In [88]:
def emotions_nrclex(texto):
    
    for i in range(len(texto)):
        text_object = NRCLex(texto[i])
        emotions = text_object.top_emotions # text_object.affect_frequencies 
                                        #diccionario con todas las emociones y sus fr.rel
        yield emotions

emotions_NRCLex = []
emotions_NRCLex_categories = []

for i in emotions_nrclex(df['texto_limpio']):
    emotions_NRCLex.append(i)
    emotions_NRCLex_category = [tag for tag,freq in i]
    emotions_NRCLex_categories.append(emotions_NRCLex_category)

df['emotions_NRCLex'] = emotions_NRCLex
#df['emotions_NRCLex_categories'] = emotions_NRCLex_categories
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,id,text,is_humor,humor_rating,humor_controversy,offense_rating,texto_limpio,sentimiento_stanza,polaridad_media_sentic,sentiwordnet_score,sentiwordnet_overall_Sentiment,emotions_NRCLex,emotions_NRCLex_categories
0,0,0,0,0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.2,tennessee we be the good state nobody even com...,0,0.089556,0.625,Positive,"[(anger, 0.125), (trust, 0.125), (surprise, 0....","[anger, trust, surprise, positive, negative, d..."
1,1,1,1,1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.1,a man insert a advertisement in the classified...,1,0.166355,0.750,Positive,"[(positive, 0.6666666666666666)]",[positive]
2,2,2,2,2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.4,how many man do it take to open a can of beer ...,0,0.088885,-0.375,Negative,"[(positive, 0.25), (sadness, 0.25), (joy, 0.25...","[positive, sadness, joy, anticipation]"
3,3,3,3,3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.0,tell my mom i hit twitter follower she point o...,1,0.032154,1.000,Positive,"[(trust, 0.4)]",[trust]
4,4,4,4,4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.1,rose be dead love be fake wedding be basically...,1,0.059833,-0.375,Negative,"[(positive, 0.25), (negative, 0.25), (sadness,...","[positive, negative, sadness, joy]"


##### Afinn score  doc:http://www2.imm.dtu.dk/pubdb/edoc/imm6006.pdf

AFINN lexicon asigna a palabras un score entre -5 a 5. Los scores negativos indican sentimientos negativos, y scores positivos indican sentimientos positivos.
Valor de cada token en la oración se suma, por lo que el valor de esos sentimientos son arbitrarios.
https://github.com/thisandagain/sentiment/issues/137#issuecomment-398951315
Variación de ANEW lexicon.

In [125]:
afinn = Afinn(language='en')
df['afinn_score'] = df['texto_limpio'].apply(afinn.score)

def tag(scores):
    sentiment = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in scores]
    return sentiment
tag = tag(df['afinn_score'])
df['afinn_score_tag'] = tag

In [1]:
import pandas as pd
df = pd.read_excel("datos_features.xlsx")
df['afinn_score'].head()

0   -1
1    2
2    0
3    1
4   -3
Name: afinn_score, dtype: int64

In [14]:
df['afinn_score'].unique()
from afinn import Afinn
afinn = Afinn(language='en')
l=['crisis','no']
afinn.score('no')+afinn.score('no')+afinn.score('no')+afinn.score('no')

-4.0

###### ANEW Score
database of 14.000 English words that have been manually rated by many human volunteers on three affective measures: pleasure (valence), arousal (excitement), and dominance (level of control),split data by gender, age, and educational differences in raters.
affective ratings are on a scale from 1 to 9, where 1 is the least pleasurable/exciting/controlling, and 9 is the most.

Analisis palabra a palabra. si una palabra tiene en sus tres palabras o menos, anteriores una negativa, se invierte el valor de polaridad.
se extrae el valor de cada palabra de cada tipo de característica , y se hace la media para las 3.

In [7]:
#https://github.com/dwzhou/SentimentAnalysis
#df_anew = pd.read_csv('https://raw.githubusercontent.com/dwzhou/SentimentAnalysis/master/lib/EnglishShortened.csv')
#df_anew.to_csv('/Users/luciainesmerlo/Desktop/TFG/HaHackaton2020/datos_anew.csv', sep=',')
df_anew = pd.read_csv('datos_anew.csv',sep=',')
#df_anew.head()

In [8]:
def detect_negative_words(words):
    #return any(word in ('not', 'n', "n't",'no') for word in words)
    return any(word in ("no", "without", "nil","not", "n't", "never", "none", "neith", "nor", "non",
                       "deny", "reject", "refuse", "subside", "retract", "non")
               for word in words)

def Average(lst):
    try:
        return sum(lst) / len(lst)
    except ZeroDivisionError:
        return 0

texto = df['texto_limpio']
anew_score_sentiment = []
anew_score_arousal = []
anew_score_dominance = []

words_sentiment = {
    row['Word']: {
        'valence': float(row['valence']),
        'arousal': float(row['arousal']),
        'dominance': float(row['dominance']),
    }
    
    for _, row in df_anew.iterrows()
}

for i, t in enumerate(texto):  

    # print('oracion',i)
    v_list = []  # holds valence scores
    a_list = []  # holds arousal scores
    d_list = []  # holds dominance scores
    found_words = 0
    words = t.split()

    for index, w in enumerate(words):

        # check for negation in 3 words before current word
        neg = detect_negative_words(words[max(index-3, 0):index])

        word_sentiment = words_sentiment.get(w)
        if word_sentiment:
            found_words += 1
            v = word_sentiment['valence']  # valence (pleasure)
            a = word_sentiment['arousal']  # arousal (excitement)
            d = word_sentiment['dominance']  # dominance (level of control)

            if neg:  # reverse polarity for this word
                v = 5 - (v - 5)
                a = 5 - (a - 5)
                d = 5 - (d - 5)
            v_list.append(v)
            a_list.append(a)
            d_list.append(d)
 
    sentiment = Average(v_list)
    arousal = Average(a_list)
    dominance = Average(d_list)

    anew_score_sentiment.append(sentiment)
    anew_score_arousal.append(arousal)
    anew_score_dominance.append(dominance)

In [12]:
df['sentiment_anew']=anew_score_sentiment
df['arousal_anew']=anew_score_arousal
df['dominance_anew']=anew_score_dominance
label_anew = []

for i,value in enumerate(df['sentiment_anew']):
    
    if value > 6:
        label = 'positive'
    elif value < 4:
        label = 'negative'
    else:
        label = 'neutral'
        
    label_anew.append(label)
    
df['label_anew']=label_anew

###### Scores lexicon-abusive-words binary

In [19]:
#df_binary = pd.read_csv('https://raw.githubusercontent.com/uds-lsv/lexicon-of-abusive-words/master/Lexicons/baseLexicon.txt')
#df_binary.to_csv('/Users/luciainesmerlo/Desktop/TFG/HaHackaton2020/lexicon-of-abusive-words-binary.csv')

In [16]:
df_binary = pd.read_excel('/Users/luciainesmerlo/Desktop/TFG/HaHackaton2020/binary-lexicon-abusive-words.xlsx')

In [111]:
def tags(tag):
    if tag.startswith('adj'):
        return 'ADJ'
    elif tag.startswith('noun'):
        return 'NOUN'
    elif tag.startswith('verb'):
        return 'VERB'
    else:
        return None
# https://universaldependencies.org/u/pos/index.html

words_abusive = {
    row['word']: {
        'tag': tags(str(row['tag'])),
        'class': str(row['class'])
    }
    
    for _, row in df_binary.iterrows()
}

In [112]:
text=df['texto_limpio']

def tageo(rows):
    nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')
    for i, t in enumerate(rows):
        
        scores = []
        pos_tag = nlp(t)
        pos_tagging = {
            word.text : word.upos
            for sent in pos_tag.sentences for word in sent.words
        }
        
        yield pos_tagging
        
tagged_words = tageo(text)


In [113]:
scores = []
for index, p in enumerate(tagged_words):
    # p es un diccionario
    for k, v in p.items():
        score = 0
        info_word = words_abusive.get(k)
        
        if info_word:
            tag_in_lexicon = info_word.get('tag') 
            class_in_lexicon = info_word.get('class')
            
            if tag_in_lexicon == tag_in_lexicon and class_in_lexicon == 'True':
                score += 1
                
    scores.append(score)

2022-03-17 17:57:38 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-03-17 17:57:38 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

2022-03-17 17:57:38 INFO: Use device: cpu
2022-03-17 17:57:38 INFO: Loading: tokenize
2022-03-17 17:57:38 INFO: Loading: pos
2022-03-17 17:57:38 INFO: Done loading processors!


In [115]:
df['abusive_scores_binary']=scores

##### Scores lexicon-abusive-words expanded

dado este lexicon, que contiene palabras, su tipo y su score, se computa para cada palabra en las lineas su score (se considera si esa palabra con el mismo tipo está en ambos datasets) para posteriormente calcular el valor medio de la oración. 

In [7]:
#df_abusive = pd.read_csv('https://raw.githubusercontent.com/uds-lsv/lexicon-of-abusive-words/master/Lexicons/expandedLexicon.txt')
#df_abusive.to_csv('/Users/luciainesmerlo/Desktop/TFG/HaHackaton2020/lexicon-of-abusive-words.csv')

In [4]:
df_abusive = pd.read_excel('/Users/luciainesmerlo/Desktop/TFG/HaHackaton2020/lexicon-abusive-words.xlsx')

In [100]:
def Average(lst):
    try:
        return sum(lst) / len(lst)
    except ZeroDivisionError:
        return 0

def tags(tag):
    if tag.startswith('adj'):
        return 'ADJ'
    elif tag.startswith('noun'):
        return 'NOUN'
    elif tag.startswith('verb'):
        return 'VERB'
    else:
        return None
# https://universaldependencies.org/u/pos/index.html

words_abusive = {
    row['word']: {
        'tag': tags(row['tag']),
        'score': float(row['score'])
    }
    
    for _, row in df_abusive.iterrows()
}

text=df['texto_limpio']

def tageo(rows):
    nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')
    for i, t in enumerate(rows):
        scores = []
        pos_tag = nlp(t)
        pos_tagging = [(word.text,word.upos) for sent in pos_tag.sentences 
                       for word in sent.words]
        yield pos_tagging
        
tagged_words = tageo(text)

In [101]:
abusive_scores = []

for index, p in enumerate(tagged_words): #for each line
    scores = []
    for i, text in enumerate(p):
        
        word = text[0]
        tag_of_word = text[1]

        info_word = words_abusive.get(word)
        
        if info_word:
            
            tag_in_lexicon = info_word.get('tag')
            
            if tag_of_word == tag_in_lexicon:
                score = info_word.get('score')
            else:
                score = 0
            
            scores.append(score)
                
    s = Average(scores)
    abusive_scores.append(s)

2022-03-11 13:16:39 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-03-11 13:16:39 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

2022-03-11 13:16:39 INFO: Use device: cpu
2022-03-11 13:16:39 INFO: Loading: tokenize
2022-03-11 13:16:39 INFO: Loading: pos
2022-03-11 13:16:39 INFO: Done loading processors!


In [103]:
df['abusive_scores']=abusive_scores

In [117]:
import os
df.to_excel('/Users/luciainesmerlo/Desktop/TFG/HaHackaton2020/datos_features.xlsx')